# Treino do modelo

In [3]:
import os
import sys

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from Functions.Pipeline import pipeline_completa_skus
from Functions.FNC_Pro import lista_produtos, configurar_credenciais_bq,produtos_selecionados

configurar_credenciais_bq(r'G:Drives compartilhados/Bases BI/epoca-230913-b478a9a0dd4c.json')

# --- Caminho para lista personalizada de produtos ---

""" 
Colocar o caminho do arquivo excel (.xlsx) com a lista dos produtos que você deseja treinar

Ex: produtos_selecionado(Documents/Planilhas/Lista_produtos.xlsx)

Depois é só substituir na função abaixo lista_produtos colocando a variável SKUS = lista_selecionados

"""

lista_selecionados = produtos_selecionados('C:/Users/joao.pcarvalho/Desktop/Git Repositórios/Forecast-vendas-e-elasticidade/Lista Produtos/Lista.xlsx')


# --- Definir filtros para os produtos ---
produtos = lista_produtos(
    'G:/Drives compartilhados/Planilha de Impostos/BI/Planilha com Impostos.xlsm',
    SKUS= ['7172']
)

# --- Instruções do lista_produtos ---

""" 
Classificacao = A ou B ou C
Ativo = Sim ou Nao
SKUS = ['sku1','sku2','sku3','sku4']
SKUS = lista_selecionados

"""

# --- Executar pipeline de treinamento ---
# Esta etapa apenas treina os modelos, salva os arquivos e gera o relatório de comparação.
resultados_consolidados = pipeline_completa_skus(
    df_produtos=produtos,
    n_splits=10
)

print("\nResultados consolidados do treinamento:")
display(resultados_consolidados.head())

Encontrados 1 SKUs com os filtros: Classificação='None', Ativo='None', SKUs='['7172']'
--- INICIANDO PIPELINE DE TREINAMENTO DE MODELOS ---
Total de SKUs para processar: 1
------------------------------------------------------------

>>> Processando SKU 7172 (1/1) <<<
SKU 7172: Histórico de dados ajustado para começar em 2023-01-01, o primeiro dia com vendas.
  Modelo TSCV:
    AIC: 749.93, BIC: 766.08
    Coeficientes: [-4.92453765  0.13842693  0.1207295 ]
  Modelo ARIMAX:
    AIC: 1039.32, BIC: 1083.52
    Coeficientes: {'intercept': 2.7568193749471086, 'Log_Preco': -7.462656588594003, 'Quarta-feira': 0.055664268435965066, 'Terça-feira': 0.12781610582774683, 'ar.L1': 1.2080239971360143, 'ar.L2': -0.26769062569106605, 'ma.L1': -0.7132601299717676, 'ma.L2': 0.041761554658225654, 'sigma2': 0.17216541382040015}

--- CONSOLIDANDO RESULTADOS FINAIS ---
Relatório de modelos consolidado salvo em: ../Resultados/Relatorio_Modelos_Consolidado_20251002.csv
---------------------------------------

,sku,data_rodagem,modelo_ideal,intercepto_tscv,coef_log_preco_tscv,coef_quarta-feira_tscv,coef_terça-feira_tscv,intercepto_sarimax,coef_log_preco_sarimax,coef_quarta-feira_sarimax,coef_terça-feira_sarimax,AIC_sarimax,BIC_sarimax,AIC_cruzado,BIC_cruzado
0,7172,2025-10-02 15:42:06,TSCV,31.513087,-4.924538,0.138427,0.120729,2.756819,-7.462657,0.055664,0.127816,1039.319343,1083.5161,749.927621,766.076287


# Teste do modelo

In [ ]:
import os
import sys
import pandas as pd
from IPython.display import display
import joblib
from statsmodels.tsa.statespace.sarimax import SARIMAXResults

current_dir = os.getcwd()
project_root = os.path.abspath(os.path.join(current_dir, '..'))
if project_root not in sys.path:
    sys.path.append(project_root)

from Functions.FNC_Previsoes import prever_demanda_com_modelos_salvos, pred_prox_30_dias
from Functions.FNC_Pro import Base_venda

# --- Definir os caminhos ---
caminho_pasta_modelos = os.path.join(project_root, 'Modelos')
caminho_planilha_precos = os.path.join(project_root, 'Forecast', 'Precos_para_previsoes.xlsx')
caminho_resultados = os.path.join(project_root, 'Resultados')


# --- 1. Previsões com base no arquivo de preços ---
print("--- INICIANDO PREVISÕES COM BASE NO ARQUIVO DE PREÇOS ---")
df_previsoes_salvas = prever_demanda_com_modelos_salvos(
    caminho_pasta_modelos,
    caminho_planilha_precos
)

if not df_previsoes_salvas.empty:
    # Salvar o arquivo consolidado
    caminho_arquivo_consolidado = os.path.join(caminho_resultados, 'previsoes_consolidado.csv')
    df_previsoes_salvas.to_csv(caminho_arquivo_consolidado, index=False, sep=';', decimal=',')
    print(f"\nArquivo de previsões consolidado salvo em: {caminho_arquivo_consolidado}")
    
    print("\nAmostra das previsões geradas a partir do arquivo de preços:")
    display(df_previsoes_salvas.head())


# --- 2. Previsões de 30 dias para o futuro (com gráficos) ---
print("\n\n--- INICIANDO PREVISÕES DE 30 DIAS PARA O FUTURO ---")
try:
    df_precos = pd.read_excel(caminho_planilha_precos)
    skus_para_prever = df_precos['SKU'].astype(str).unique()
    print(f"SKUs encontrados no arquivo de preços para gerar previsões futuras: {list(skus_para_prever)}")
except FileNotFoundError:
    print(f"ERRO: Arquivo de preços não encontrado em '{caminho_planilha_precos}'")
    skus_para_prever = []

previsoes_futuras_consolidadas = []
if skus_para_prever.any():
    for sku in skus_para_prever:
        print(f"\n--- Gerando previsão de 30 dias para o SKU: {sku} ---")
        
        caminho_modelo_tscv = os.path.join(caminho_pasta_modelos, f'modelo_tscv_{sku}.joblib')
        caminho_modelo_sarimax = os.path.join(caminho_pasta_modelos, f'modelo_sarimax_{sku}.pkl')

        if not os.path.exists(caminho_modelo_tscv) or not os.path.exists(caminho_modelo_sarimax):
            print(f"  AVISO: Modelos para o SKU {sku} não encontrados. Pulando.")
            continue
        
        modelo_tscv = joblib.load(caminho_modelo_tscv)
        modelo_sarimax = SARIMAXResults.load(caminho_modelo_sarimax)
        Venda = Base_venda(sku)
        
        if Venda.empty:
            print(f"  AVISO: Não foi possível carregar o histórico de vendas para o SKU {sku}. Pulando.")
            continue

        df_previsao_futura = pred_prox_30_dias(
            modelo_tscv,
            modelo_sarimax,
            Venda,
            sku,
            X_cols_tscv=['Log_Preco', 'Quarta-feira', 'Terça-feira']
        )
        previsoes_futuras_consolidadas.append(df_previsao_futura)

if previsoes_futuras_consolidadas:
    df_previsoes_finais = pd.concat(previsoes_futuras_consolidadas, ignore_index=True)
    
    # Salvar o arquivo consolidado de previsões futuras
    caminho_futuro_consolidado = os.path.join(caminho_resultados, 'previsoes_futuras_consolidado.csv')
    df_previsoes_finais.to_csv(caminho_futuro_consolidado, index=False, sep=';', decimal=',')
    print(f"\nArquivo de previsões futuras consolidado salvo em: {caminho_futuro_consolidado}")

    print("\nAmostra das previsões de 30 dias consolidadas:")
    display(df_previsoes_finais.head())
else:
    print("\nNenhuma previsão de 30 dias foi gerada.")